In [4]:
import pandas as pd
import altair as alt

# Load the data
kia_boys = pd.read_csv("../../data/clean/google-trends.csv")

# Pivot the data
kia_boys = kia_boys.melt(id_vars="week", var_name="trend", value_name="searches")
kia_boys["trend"] = kia_boys["trend"].apply(lambda x: x.title())

# Map colors to the trends
colors = {
    "Kia Boys": "#A8C4FF",
    "Hyundai Theft": "#A56A6E",
    "Kia Theft": "#89B587"
}

kia_boys["color"] = kia_boys["trend"].map(colors)

cols = kia_boys["trend"].unique()

## Visualize the data in Altair
line = alt.Chart(
    kia_boys,
    title = alt.Title(
        "Google Searches over Time for Car Theft Terms",
        subtitle = ["Searches for Kia Thefts, Hyundai Thefts, and Kia Boys in the US, 2020-2024"]
    )
    ).mark_line().encode(
    x=alt.X("week:T").title("Week"),
    y=alt.Y("searches:Q").title("Relative Search Popularity"),
    color=alt.Color("trend:N").scale(domain=list(colors.keys()), range=list(colors.values())).title("Search Term"),
)

nearest = alt.selection_point(
    nearest=True, 
    on="pointerover",
    fields=["week"],
    empty=False
)

points = line.mark_point(filled=True).encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
    size = alt.Size("searches:Q", scale=alt.Scale(range=[10, 800]), legend=None),
)

rules = alt.Chart(kia_boys).mark_rule(color="gray").encode(
    x="week:T",
).transform_filter(
    nearest
)

rules = alt.Chart(kia_boys).transform_pivot(
    "trend",
    value="searches",
    groupby=["week"]
).mark_rule(color="gray").encode(
    x="week:T",
    opacity=alt.condition(nearest, alt.value(0.3), alt.value(0)),
    tooltip=[alt.Tooltip(c, type="quantitative") for c in cols],
).add_params(nearest)

alt.layer(
    line, points, rules
).properties(
    width=625, height=400
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=15
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22
).configure_legend(
    titleFontSize=15,
    labelFontSize=12,
).configure_line(
    strokeWidth=1.5,
).configure_point(
    size=50
).configure_title(
    font='Arial',
    fontSize=22
)

alt.LayerChart(...)